# Overview
- nb002を1つのサイトだけで学習してみる

In [1]:
import subprocess
cmd = "git rev-parse --short HEAD"
hash = subprocess.check_output(cmd.split()).strip().decode('utf-8')
print(hash)

1c30b36


# Const

In [2]:
NB = '025'
SITE = '5c3c44b80379370013e0fd2b'
DIR_TRAIN = './../data_ignore/input/train/'
DIR_TEST = './../data_ignore/input/test/'
DIR_WIFI = './../data_ignore/input/wifi/'
PATH_SUB = './../data_ignore/input/sample_submission.csv'
PATH_99_SUB = './../data/input/floor_99per_acc_sub.csv'
DIR_SAVE_IGNORE = f'./../data_ignore/nb/{NB}/'
DIR_SAVE = f'./../data/nb/{NB}/'

In [3]:
config_str = '''
globals:
    seed: 5713
    device: cuda
    n_label: 24
    n_splits: 5
    random_sate: 45
    lr: 0.005
    patience: 10
    epoch: 100
    batch_size: 512
    skip_evaluate_num: 5
    num_feats: 20
    t_mux: 10
'''

# Import everything I need:)

In [4]:
import os
import yaml
import types
import random
import pickle
import builtins
import numpy as np
import pandas as pd
from icecream import ic
# from tqdm import tqdm
from fastprogress import progress_bar, master_bar
from glob import glob
from loguru import logger
from collections import OrderedDict

# sklearn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import KFold

# pytorch
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F

# Function

In [5]:
def imports():
    for name, val in globals().items():
        # module imports
        if isinstance(val, types.ModuleType):
            yield name, val

            # functions / callables
        if hasattr(val, '__call__'):
            yield name, val


def noglobal(f):
    '''
    ref: https://gist.github.com/raven38/4e4c3c7a179283c441f575d6e375510c
    '''
    return types.FunctionType(f.__code__,
                              dict(imports()),
                              f.__name__,
                              f.__defaults__,
                              f.__closure__
                              )


def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

# Preparation

load config

In [6]:
config = yaml.safe_load(config_str)
config

{'globals': {'seed': 5713,
  'device': 'cuda',
  'n_label': 24,
  'n_splits': 5,
  'random_sate': 45,
  'lr': 0.005,
  'patience': 10,
  'epoch': 100,
  'batch_size': 512,
  'skip_evaluate_num': 5,
  'num_feats': 20,
  't_mux': 10}}

<br>

set

In [7]:
seed_everything(config['globals']['seed'])

if not os.path.exists(DIR_SAVE_IGNORE):
    os.makedirs(DIR_SAVE_IGNORE)
if not os.path.exists(DIR_SAVE):
    os.makedirs(DIR_SAVE)

<br>

load dataset

In [8]:
with open(f'{DIR_WIFI}train_all.pkl', 'rb') as f:
    df_train = pickle.load( f)
with open(f'{DIR_WIFI}test_all.pkl', 'rb') as f:
    df_test = pickle.load( f)

<br>

site絞る

In [9]:
df_train = df_train[df_train.site_id == SITE]
df_test = df_test[df_test.site_id == SITE]

<br>

preprocessing

In [10]:
bssid_feats = [f'bssid_{i}' for i in range(config['globals']['num_feats'])]
rssi_feats  = [f'rssi_{i}' for i in range(config['globals']['num_feats'])]

In [11]:
# bssidの一覧作成
# wifi_bassidにはtrainとtest両方のbssidの一覧が含まれる

wifi_bssids = []
for i in range(100):
    wifi_bssids.extend(df_train.iloc[:,i].values.tolist())
wifi_bssids = list(set(wifi_bssids))

wifi_bssids_size = len(wifi_bssids)
print(f'BSSID TYPES: {wifi_bssids_size}')

wifi_bssids_test = []
for i in range(100):
    wifi_bssids_test.extend(df_test.iloc[:,i].values.tolist())
wifi_bssids_test = list(set(wifi_bssids_test))

wifi_bssids_size = len(wifi_bssids_test)
print(f'BSSID TYPES: {wifi_bssids_size}')

wifi_bssids.extend(wifi_bssids_test)
wifi_bssids_size = len(wifi_bssids)
print(f'BSSID TYPES: {wifi_bssids_size}')

BSSID TYPES: 2747
BSSID TYPES: 489
BSSID TYPES: 3236


In [12]:
# LabelEncoding & StandardScaler

le = LabelEncoder()
le.fit(wifi_bssids)
le_site = LabelEncoder()
le_site.fit(df_train['site_id'])

ss = StandardScaler()
ss.fit(df_train.loc[:, rssi_feats])

StandardScaler()

In [13]:
df_train.loc[:, rssi_feats] = ss.transform(df_train.loc[:, rssi_feats])
df_test.loc[:, rssi_feats] = ss.transform(df_test.loc[:, rssi_feats])
for feat in bssid_feats:
    df_train.loc[:, feat] = le.transform(df_train.loc[:, feat])
    df_test.loc[:, feat] = le.transform(df_test.loc[:, feat])
    
    df_train.loc[:, feat] = df_train.loc[:, feat] + 1
    df_test.loc[:, feat] = df_test.loc[:, feat] + 1
    
df_train.loc[:, 'site_id'] = le_site.transform(df_train.loc[:, 'site_id'])
df_test.loc[:, 'site_id'] = le_site.transform(df_test.loc[:, 'site_id'])

df_train.loc[:, rssi_feats] = ss.transform(df_train.loc[:, rssi_feats])
df_test.loc[:, rssi_feats] = ss.transform(df_test.loc[:, rssi_feats])

In [14]:
site_count = len(df_train['site_id'].unique())
df_train.reset_index(drop=True, inplace=True)

In [15]:
df_train.head()

,bssid_0,bssid_1,bssid_2,bssid_3,bssid_4,bssid_5,bssid_6,bssid_7,bssid_8,bssid_9,...,rssi_95,rssi_96,rssi_97,rssi_98,rssi_99,x,y,floor,path,site_id
0,2231,119,1878,2689,1191,1853,903,2221,1070,2627,...,-65,-65,-65,-66,-66,258.99582,107.52311,-1,5d8c805cba656a000636d4aa,0
1,119,2231,1853,1010,1878,2627,2689,208,998,2596,...,-65,-65,-66,-66,-66,258.99582,107.52311,-1,5d8c805cba656a000636d4aa,0
2,119,2231,1853,2627,208,1676,1191,1333,567,2221,...,-65,-65,-65,-65,-65,263.35950,117.29801,-1,5d8c805cba656a000636d4aa,0
3,2627,1853,2231,119,960,1787,2221,1172,1461,413,...,-66,-66,-66,-66,-66,263.35950,117.29801,-1,5d8c805cba656a000636d4aa,0
4,1853,2627,2231,119,903,2067,2400,2346,2349,2046,...,-66,-67,-67,-67,-67,263.35950,117.29801,-1,5d8c805cba656a000636d4aa,0


# Execute

In [16]:
class EarlyStopping:
    """
    Early stops the training if validation loss doesn't improve after a given patience.
    based on: https://github.com/Bjarten/early-stopping-pytorch
    """
    def __init__(self, save_name, fold, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.fold = fold
        self.save_name = save_name

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            logger.info(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            logger.info(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
            
        torch.save(model.state_dict(),  f'{self.save_name}_{self.fold}.pt')
        self.val_loss_min = val_loss

In [17]:
class IndoorWiFiNet(nn.Module):
    def __init__(self, bssid_size, site_size, num_feats):
        super(IndoorWiFiNet, self).__init__()
        self.bssid_embedding = nn.Embedding(bssid_size, 64, max_norm=True)
        self.site_embedding = nn.Embedding(site_size, 64, max_norm=True)

        self.rssi = nn.Sequential(
            nn.BatchNorm1d(20),
            nn.Linear(20, num_feats * 64)
        )

        concat_size = 64 + (num_feats * 64) + (num_feats * 64)
        self.bn1 = nn.BatchNorm1d(concat_size)

        self.flatten = nn.Flatten()

        self.dropout1 = nn.Dropout(0.3)
        self.linear1 = nn.Linear(in_features=concat_size, out_features=256)#, bias=False)
        self.bn2 = nn.BatchNorm1d(256)

        self.linear2 = nn.Linear(in_features=256, out_features=128)#, bias=False)
        self.linear3 = nn.Linear(in_features=128, out_features=16)#, bias=False)

        self.bn3 = nn.BatchNorm1d(128)
        self.bn4 = nn.BatchNorm1d(16)


        self.xy = nn.Linear(in_features=16, out_features=2)#, bias=False)
        self.floor = nn.Linear(in_features=16, out_features=1)#, bias=False)
        

    def forward(self, bssid, rssi, site):

        site = torch.reshape(site, (-1, 1))

        bssid_out = self.bssid_embedding(bssid)
        site_out = self.site_embedding(site)

        rssi_out = self.rssi(rssi)

        bssid_out = self.flatten(bssid_out)
        site_out = self.flatten(site_out)

        x = torch.cat([bssid_out, rssi_out, site_out], dim=1)
        x = self.bn1(x)
        x = self.dropout1(x)
        x = F.relu(self.linear1(x))
        x = self.bn2(x)

        x = F.relu(self.linear2(x))
        x = self.bn3(x)

        x = F.relu(self.linear3(x))
        x = self.bn4(x)

        #x = self.lstm(x)

        xy = self.xy(x)
        floor = self.floor(x)

        return xy, floor

In [18]:
class IndoorWiFiDataSet(Dataset):
    def __init__(self, wifi_df, bssid_feats_, rssi_feats_, data_type='test'):
        self.wifi_df = wifi_df
        self.data_type = data_type
        self.bssids = wifi_df[bssid_feats_].to_numpy().astype(np.int)    # <----- リファクタ
        self.rssis = wifi_df[rssi_feats_].to_numpy().astype(np.float)    # <----- リファクタ
        self.site_ids = wifi_df['site_id'].to_numpy()
        if data_type in ['train', 'valid']:
            self.floors = wifi_df['floor'].to_numpy().astype(np.int)
            self.xs = wifi_df['x'].to_numpy()
            self.ys = wifi_df['y'].to_numpy()
        
    def __len__(self):
        return len(self.wifi_df)

    def __getitem__(self, idx):

        #wifi_row = self.wifi_df.iloc[idx]

        bssid = self.bssids[idx,:]
        rssi = self.rssis[idx,:]
        site_id = self.site_ids[idx]

        if self.data_type in ['test', 'valid']:
            return bssid, rssi, site_id

        elif self.data_type == 'train':
            x = self.xs[idx]
            y = self.ys[idx]
            floor = self.floors[idx]

            return bssid, rssi, site_id, x, y, floor


In [19]:
class IndoorLoss(nn.Module):
    def __init__(self):
        super().__init__()

        self.mse = nn.MSELoss()

    def forward(self, x, y, floor, x_pred, y_pred, floor_pred):

        x_loss = self.mse(x, x_pred)
        y_loss = self.mse(y, y_pred)
        floor_loss = self.mse(floor, floor_pred)

        return x_loss + y_loss + floor_loss

In [20]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device(config['globals']['device'])

In [21]:
%%time

oof_x = np.zeros((len(df_train)))
oof_y = np.zeros((len(df_train)))
oof_floor = np.zeros((len(df_train)))
cv_scores = {}

kf = KFold(n_splits=config['globals']['n_splits'], random_state=42, shuffle=True)
for fold, (train_index, valid_index) in enumerate(kf.split(df_train)):
    print(f'\r\n====== {fold + 1} ======')
    net = IndoorWiFiNet(wifi_bssids_size, site_count, config['globals']['num_feats'])
    net = net.to(device)

    #criterion = IndoorLoss()
    criterion = nn.MSELoss()
    criterion = criterion.to(device)
    #optimizer = optim.SGD(net.parameters(), lr=Config.LR, weight_decay=0.0001, momentum=0.9)
    optimizer = torch.optim.Adam(net.parameters(), lr=config['globals']['lr'])
    
    train, valid = df_train.iloc[train_index], df_train.iloc[valid_index]

    # TODO oof用のdataloaderはいらないはず
    train_dataset = IndoorWiFiDataSet(train.reset_index(drop=True), bssid_feats, rssi_feats, data_type='train')
    valid_dataset = IndoorWiFiDataSet(valid.reset_index(drop=True), bssid_feats, rssi_feats, data_type='valid')

    trainloader = DataLoader(train_dataset, batch_size=config['globals']['batch_size'], shuffle=True, drop_last=True, num_workers=4)
    validloader = DataLoader(valid_dataset, batch_size=config['globals']['batch_size'], num_workers=4)

    early_stopping = EarlyStopping(save_name=f'{DIR_SAVE_IGNORE}checkpoint_{NB}', fold=fold+1, patience=config['globals']['patience'], verbose=True)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config['globals']['t_mux'])


    ############# TRAIN #############
    val_losses = []
    val_metrics = []
    mb = master_bar(range(1, config['globals']['epoch']+1))
    for epoch in mb:
        mb.main_bar.comment = f'epoch = {epoch}'
        running_loss = 0.0
        #train_lwlrap = 0.0
#         n_iter = len(trainloader)
#         with tqdm(enumerate(trainloader), total=n_iter) as pbar:
        for i, (bssid, rssi, site_id, x, y, floor) in enumerate(progress_bar(trainloader, parent=mb)):
            net.train()
            # zero the parameter gradients
            optimizer.zero_grad()

            bssid, rssi, site_id, x, y, floor = bssid.to(device).long(), rssi.to(device).float(), site_id.to(device).long(), x.to(device).float(), y.to(device).float(), floor.to(device)

            xy_pred, floor_pred = net(bssid, rssi, site_id)
            xy_pred = xy_pred.transpose(0, 1).squeeze(-2).reshape(-1)
            label = torch.cat([x, y], dim=-1)
            loss = criterion(xy_pred, label)
            #loss = criterion(x, y, floor, x_pred, y_pred, floor_pred)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
#             mb.child.comment(OrderedDict(
#                 epoch="{:>10}".format(epoch), loss="{:.4f}".format(loss.item())
#             ))
            mb.child.comment = f'loss={loss.item():.4f}'
        scheduler.step()
            
        #if epoch < Config.SKIP_EVALUATE_NUM:
        #    continue

        ############# VALID #############
        val_loss = 0.0
        val_lwlrap = 0.0
        x_preds = np.array([])
        y_preds = np.array([])
        val_preds_frame = []
        n_iter_val = len(validloader)
#         for i, (bssid, rssi, site_id) in tqdm(enumerate(validloader), total=len(validloader)):
        for i, (bssid, rssi, site_id) in enumerate(progress_bar(validloader, parent=mb)):
            mb.child.comment = 'calc valid'
            net.eval()

            with torch.no_grad():
                
                #melspec, labels = melspec.to(device).float(), labels.to(device).float()
                bssid, rssi, site_id = bssid.to(device).long(), rssi.to(device).float(), site_id.to(device).long()
                xy_pred, floor_pred = net(bssid, rssi, site_id)

                xy_pred = xy_pred.to('cpu').detach().numpy().copy()
                x_pred, y_pred = np.hsplit(xy_pred, 2)
                x_preds = np.concatenate([x_preds, x_pred.reshape(-1)])
                y_preds = np.concatenate([y_preds, y_pred.reshape(-1)])

        score = comp_metric(x_preds, y_preds, 0, valid['x'], valid['y'], 0)

        early_stopping(score, net)

        if early_stopping.early_stop:
            logger.info("Early stopping")
            cv_scores[f'cv{fold + 1}'] = early_stopping.best_score
            break
                
    _x_oof = np.array([])
    _y_oof = np.array([])
    ############# OOF #############
    oof_preds = []
#     for i, (bssid, rssi, site_id) in tqdm(enumerate(validloader), total=len(validloader)):
    for i, (bssid, rssi, site_id) in enumerate(progress_bar(validloader, parent=mb)):
#         net.load_state_dict(torch.load(ROOT_DIR / Path('output') / f'checkpoint_{NB}_{fold}.pt'))
        net.load_state_dict(torch.load(f'{DIR_SAVE_IGNORE}checkpoint_{NB}_{fold + 1}.pt'))
        net.eval()

        with torch.no_grad():

            bssid, rssi, site_id, x, y, floor = bssid.to(device).long(), rssi.to(device).float(), site_id.to(device).long(), x.to(device).float(), y.to(device).float(), floor.to(device)

            xy_pred, floor_pred = net(bssid, rssi, site_id)

            xy_pred = xy_pred.to('cpu').detach().numpy().copy()
            x_pred, y_pred = np.hsplit(xy_pred, 2)
            _x_oof = np.concatenate([_x_oof, x_pred.reshape(-1)])
            _y_oof = np.concatenate([_y_oof, y_pred.reshape(-1)])

    print(comp_metric(_x_oof, _y_oof, 0, valid['x'], valid['y'], 0))

    oof_x[valid_index] = _x_oof
    oof_y[valid_index] = _y_oof

    del net
    torch.cuda.empty_cache()


====== 1 ======


2021-04-09 12:34:26.863 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (inf --> 252.078050).  Saving model ...
2021-04-09 12:34:27.332 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (252.078050 --> 250.244216).  Saving model ...
2021-04-09 12:34:27.809 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (250.244216 --> 249.994945).  Saving model ...
2021-04-09 12:34:28.292 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (249.994945 --> 249.497974).  Saving model ...
2021-04-09 12:34:28.763 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (249.497974 --> 248.330045).  Saving model ...
2021-04-09 12:34:29.224 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (248.330045 --> 247.263640).  Saving model ...
2021-04-09 12:34:29.689 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (247.263640 --> 246.603270).  Saving model ...
2021-04-09 12:34:30.161 | I

13.190884952040157

====== 2 ======


2021-04-09 12:35:13.857 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (inf --> 246.392471).  Saving model ...
2021-04-09 12:35:14.355 | INFO     | __main__:__call__:35 - EarlyStopping counter: 1 out of 10
2021-04-09 12:35:14.851 | INFO     | __main__:__call__:35 - EarlyStopping counter: 2 out of 10
2021-04-09 12:35:15.361 | INFO     | __main__:__call__:35 - EarlyStopping counter: 3 out of 10
2021-04-09 12:35:15.862 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (246.392471 --> 246.343898).  Saving model ...
2021-04-09 12:35:16.371 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (246.343898 --> 245.236064).  Saving model ...
2021-04-09 12:35:16.884 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (245.236064 --> 244.452415).  Saving model ...
2021-04-09 12:35:17.411 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (244.452415 --> 243.849358).  Saving model ...
2021-04-09 12:35:17.94

12.36859490620594

====== 3 ======


...

12.952951365709541

====== 4 ======


2021-04-09 12:36:57.884 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (inf --> 251.535339).  Saving model ...
2021-04-09 12:36:58.349 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (251.535339 --> 250.643578).  Saving model ...
2021-04-09 12:36:58.823 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (250.643578 --> 248.796005).  Saving model ...
2021-04-09 12:36:59.308 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (248.796005 --> 248.193688).  Saving model ...
2021-04-09 12:36:59.787 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (248.193688 --> 246.964727).  Saving model ...
2021-04-09 12:37:00.261 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (246.964727 --> 245.983274).  Saving model ...
2021-04-09 12:37:00.736 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (245.983274 --> 245.377807).  Saving model ...
2021-04-09 12:37:01.196 | I

14.737289794192066

====== 5 ======


2021-04-09 12:37:45.389 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (inf --> 248.889386).  Saving model ...
2021-04-09 12:37:45.884 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (248.889386 --> 247.987207).  Saving model ...
2021-04-09 12:37:46.372 | INFO     | __main__:__call__:35 - EarlyStopping counter: 1 out of 10
2021-04-09 12:37:46.871 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (247.987207 --> 247.397197).  Saving model ...
2021-04-09 12:37:47.393 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (247.397197 --> 246.294626).  Saving model ...
2021-04-09 12:37:47.885 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (246.294626 --> 245.768341).  Saving model ...
2021-04-09 12:37:48.385 | INFO     | __main__:save_checkpoint:46 - Validation loss decreased (245.768341 --> 245.535593).  Saving model ...
2021-04-09 12:37:48.936 | INFO     | __main__:save_checkpoint:46 - Valid

14.008922163016356
CPU times: user 1min 4s, sys: 2min 5s, total: 3min 9s
Wall time: 4min 18s


In [23]:
oof_score = comp_metric(oof_x, oof_y, np.zeros(len(df_train)), df_train['x'], df_train['y'], np.zeros(len(df_train)))
oof_score

13.451590608358291